In [1]:
import numba
import numpy as np
import time

In [3]:
k = 20
r = np.load('cube-full-460-512-512.npy')[::k,::k,::k]
# r = np.load('../3d_numpy_array_reduced-58-64-64.npy')
ro  = 1e-5 + 1.24e-3*r - 2.83e-7*r*r + 2.79e-11*r*r*r
c = (ro + 0.112) * 1.38e-6

# тут не факт что все 1
l = 1 # dt
h = 1 # dx = dy = dz
K = l / h * c
K2 = K**2
K_2_by_3 = K**2 / 3
K2_3_ro = K2 / 3 / ro


# initial conditions
# P_pp = np.zeros_like(ro) # previous previous t - 2
# P_p  = np.zeros_like(ro) # previous          t - 1
# P    = np.zeros_like(ro) # current           t


N = 16
S = 16 # first test simple case when N == S
P_pp = np.random.random(ro.shape) # previous previous t - 2
P_p  = np.random.random(ro.shape) # previous          t - 1
P    = np.zeros(ro.shape, dtype=np.float64) # current           t

In [59]:
def old_slow1(P_pp, P_p, P, ro):
    '''slow explicit n^3, c-style implementation v0.2'''
    
    S = P_p.shape[0]
    N = P_p.shape[1]

    P[2:-2, 2:-2, 2:-2] = (2 - 7.5 * K2[2:-2, 2:-2, 2:-2]) * P_p[2:-2, 2:-2, 2:-2] - P_pp[2:-2, 2:-2, 2:-2]


    for i in range(2, S - 2):
        for j in range(2, N - 2):
            for k in range(2, N - 2):
                s1 = (P_p[i + 1, j    , k    ] + \
                      P_p[i - 1, j    , k    ] + \
                      P_p[i    , j + 1, k    ] + \
                      P_p[i    , j - 1, k    ] + \
                      P_p[i    , j    , k + 1] + \
                      P_p[i    , j    , k - 1]
                )
                P[i, j, k] += 4/3 * K2[i, j, k] * s1
                s2 = (P_p[i + 2, j    , k    ] + \
                      P_p[i - 2, j    , k    ] + \
                      P_p[i    , j + 2, k    ] + \
                      P_p[i    , j - 2, k    ] + \
                      P_p[i    , j    , k + 2] + \
                      P_p[i    , j    , k - 2]
                )
                P[i, j, k] -= K2[i, j, k] / 12 * s2

                s3 = ((P_p[i + 1, j    , k    ] - P_p[i-1 , j  , k  ]) - (P_p[i + 2, j, k] - P_p[i - 2, j, k]) / 8) * (ro[i + 1, j, k] - ro[i - 1, j, k])
                s4 = ((P_p[i    , j + 1, k    ] - P_p[i   , j-1, k  ]) - (P_p[i, j + 2, k] - P_p[i, j - 2, k]) / 8) * (ro[i, j + 1, k] - ro[i, j - 1, k])
                s5 = ((P_p[i    , j    , k + 1] - P_p[i-1 , j  , k-1]) - (P_p[i, j, k + 2] - P_p[i, j, k - 2]) / 8) * (ro[i, j, k + 1] - ro[i, j, k - 1])
                P -= K2_3_ro[i, j, k] * s3
                P -= K2_3_ro[i, j, k] * s4
                P -= K2_3_ro[i, j, k] * s5
    P[ro < 0.1] = 0
    return P

In [81]:
# @numba.jit(nopython=True, parallel=True, fastmath = True, nogil=True)
# @numba.jit()
@numba.jit(nopython=True, parallel=True, fastmath = True, nogil=True)
def fresh(P_pp, P_p, P, ro):
    '''slow explicit n^3, c-style implementation v0.2'''
    
    S = P_p.shape[0]
    N = P_p.shape[1]
    P[2:-2, 2:-2, 2:-2] = (2 - 7.5 * K2[2:-2, 2:-2, 2:-2]) * P_p[2:-2, 2:-2, 2:-2] - P_pp[2:-2, 2:-2, 2:-2]

    # numba.prange
    for i in range(2, S - 2):
        for j in range(2, N - 2):
            for k in range(2, N - 2):
                if ro[i, j, k] < 0.1:
                    P[i, j, k] = 0
                else:
                    P[i, j, k] += K2[i, j, k] * 4/3 * (
                        P_p[i + 1, j    , k    ] +
                        P_p[i - 1, j    , k    ] +
                        P_p[i    , j + 1, k    ] +
                        P_p[i    , j - 1, k    ] +
                        P_p[i    , j    , k + 1] +
                        P_p[i    , j    , k - 1]
                    ) - K2[i, j, k] / 12 * (
                        P_p[i + 2, j    , k    ] +
                        P_p[i - 2, j    , k    ] +
                        P_p[i    , j + 2, k    ] +
                        P_p[i    , j - 2, k    ] +
                        P_p[i    , j    , k + 2] +
                        P_p[i    , j    , k - 2]
                    ) - K2_3_ro[i, j, k] * (
                        ((P_p[i + 1, j    , k    ] - P_p[i-1 , j  , k  ]) - (P_p[i + 2, j, k] - P_p[i - 2, j, k]) / 8) * (ro[i + 1, j, k] - ro[i - 1, j, k]) +
                        ((P_p[i    , j + 1, k    ] - P_p[i   , j-1, k  ]) - (P_p[i, j + 2, k] - P_p[i, j - 2, k]) / 8) * (ro[i, j + 1, k] - ro[i, j - 1, k]) +
                        ((P_p[i    , j    , k + 1] - P_p[i-1 , j  , k-1]) - (P_p[i, j, k + 2] - P_p[i, j, k - 2]) / 8) * (ro[i, j, k + 1] - ro[i, j, k - 1])
                    )
    return P

In [ ]:
P = np.zeros(ro.shape, dtype=np.float64)
t0 = time.time()
a0 = old_slow1(P_pp, P_p, P, ro)
print(time.time() - t0)

P = np.zeros(ro.shape, dtype=np.float64)
t0 = time.time()
a1 = fresh(P_pp, P_p, P, ro)
print(time.time() - t0)

assert np.allclose(a0, a1)